In [1]:
import pyexasol
import pandas as pd
import boto3
import time
import io
import numpy as np
from matplotlib import pyplot as plt


dwh_host = '***'
dwh_port = '8563'
dwh_user = '***'
dwh_pass = '***'

#connect to db
dwh_conn = pyexasol.connect(dsn=dwh_host + ':' + dwh_port, user=dwh_user, password=dwh_pass)

## data function to get iterator data

In [3]:
#get iterator month data
def get_feature_types():

    #get iterator datam
    
    v_sql = """
SELECT distinct
        feature_type
from
    betting_mart.ml_poisson_features
where
    full_time_home_goals is not null
    and home_shots_for_ema is not null
    and away_shots_against_ema is not null
    """
    
    return dwh_conn.export_to_pandas(v_sql)


#get iterator month data
def get_iterator_data(feature_type_filter):

    #get iterator datam
    
    v_sql = """SELECT distinct
        to_char(match_date,'yyyymm') iterator
    FROM 
        betting_mart.ml_poisson_features
    WHERE
        --2014_15 season dropped for stable features
        season <> '2014_2015'
        --2015_16 is first training season
        AND season <> '2015_2016' and
        --feature type filter
        feature_type = '""" + str(feature_type_filter) + """' and
        --ohne null values
        home_goals_for_ema is not null and
        home_goals_against_ema is not null and
        away_goals_for_ema is not null and
        away_goals_against_ema is not null and
        home_xg_for_ema is not null and
        home_xg_against_ema is not null and
        away_xg_for_ema is not null and
        away_xg_against_ema is not null and
        home_shots_for_ema is not null and
        home_shots_against_ema is not null and
        away_shots_for_ema is not null and
        away_shots_against_ema is not null and
        home_sot_for_ema is not null and
        home_sot_against_ema is not null and
        away_sot_for_ema is not null and
        away_sot_against_ema is not null and
        home_corners_for_ema is not null and
        home_corners_against_ema is not null and
        away_corners_for_ema is not null and
        away_corners_against_ema is not null and
        home_deep_for_ema is not null and
        home_deep_against_ema is not null and
        away_deep_for_ema is not null and
        away_deep_against_ema is not null and
        home_ppda_for_ema is not null and
        home_ppda_against_ema is not null and
        away_ppda_for_ema is not null and
        away_ppda_against_ema is not null and
        full_time_home_goals is not null and
        full_time_away_goals is not null
    ORDER BY 1"""

    return dwh_conn.export_to_pandas(v_sql)


def get_home_training_data(feature_type_filter, month_filter):

    v_sql = """
select
    round(home_xg_for_ema,1)               home_xg_for_ema,    
    round(away_xg_against_ema,1)           away_xg_against_ema,    
    full_time_home_goals                   full_time_home_goals
from
    betting_mart.ml_poisson_features
where
    --filter for iterator
    to_char(match_date,'yyyymm') < """ + str(month_filter) + """ and
    feature_type = '""" + str(feature_type_filter) + """' and    
    home_xg_for_ema is not null and
    home_xg_against_ema is not null and
    away_xg_for_ema is not null and
    away_xg_against_ema is not null and
    home_shots_for_ema is not null and
    home_shots_against_ema is not null and
    away_shots_for_ema is not null and
    away_shots_against_ema is not null and
    home_sot_for_ema is not null and
    home_sot_against_ema is not null and
    away_sot_for_ema is not null and
    away_sot_against_ema is not null and
    home_corners_for_ema is not null and
    home_corners_against_ema is not null and
    away_corners_for_ema is not null and
    away_corners_against_ema is not null and
    home_deep_for_ema is not null and
    home_deep_against_ema is not null and
    away_deep_for_ema is not null and
    away_deep_against_ema is not null and
    home_ppda_for_ema is not null and
    home_ppda_against_ema is not null and
    away_ppda_for_ema is not null and
    away_ppda_against_ema is not null and
    full_time_home_goals is not null and
    full_time_away_goals is not null
    """
    
    
    return dwh_conn.export_to_pandas(v_sql)


def get_away_training_data(feature_type_filter, month_filter):

    v_sql = """
select
    round(home_xg_against_ema,1)           home_xg_against_ema,
    round(away_xg_for_ema,1)               away_xg_for_ema,
    full_time_away_goals              full_time_away_goals
from
    betting_mart.ml_poisson_features
where
    --filter for iterator
    to_char(match_date,'yyyymm') < """ + str(month_filter) + """ and
    feature_type = '""" + str(feature_type_filter) + """' and
    home_goals_for_ema is not null and
    home_goals_against_ema is not null and
    away_goals_for_ema is not null and
    away_goals_against_ema is not null and
    home_xg_for_ema is not null and
    home_xg_against_ema is not null and
    away_xg_for_ema is not null and
    away_xg_against_ema is not null and
    home_shots_for_ema is not null and
    home_shots_against_ema is not null and
    away_shots_for_ema is not null and
    away_shots_against_ema is not null and
    home_sot_for_ema is not null and
    home_sot_against_ema is not null and
    away_sot_for_ema is not null and
    away_sot_against_ema is not null and
    home_corners_for_ema is not null and
    home_corners_against_ema is not null and
    away_corners_for_ema is not null and
    away_corners_against_ema is not null and
    home_deep_for_ema is not null and
    home_deep_against_ema is not null and
    away_deep_for_ema is not null and
    away_deep_against_ema is not null and
    home_ppda_for_ema is not null and
    home_ppda_against_ema is not null and
    away_ppda_for_ema is not null and
    away_ppda_against_ema is not null and
    full_time_home_goals is not null and
    full_time_away_goals is not null
    """
    
    
    return dwh_conn.export_to_pandas(v_sql)



def get_home_prediction_data(feature_type_filter, month_filter):
    
    v_sql = """
select
    football_match_his_lid,
    round(home_xg_for_ema,1)               home_xg_for_ema,    
    round(away_xg_against_ema,1)           away_xg_against_ema
from
    betting_mart.ml_poisson_features
where
    --filter for iterator
    to_char(match_date,'yyyymm') = """ + str(month_filter) + """ and
    feature_type = '""" + str(feature_type_filter) + """' and
    home_goals_for_ema is not null and
    home_goals_against_ema is not null and
    away_goals_for_ema is not null and
    away_goals_against_ema is not null and
    home_xg_for_ema is not null and
    home_xg_against_ema is not null and
    away_xg_for_ema is not null and
    away_xg_against_ema is not null and
    home_shots_for_ema is not null and
    home_shots_against_ema is not null and
    away_shots_for_ema is not null and
    away_shots_against_ema is not null and
    home_sot_for_ema is not null and
    home_sot_against_ema is not null and
    away_sot_for_ema is not null and
    away_sot_against_ema is not null and
    home_corners_for_ema is not null and
    home_corners_against_ema is not null and
    away_corners_for_ema is not null and
    away_corners_against_ema is not null and
    home_deep_for_ema is not null and
    home_deep_against_ema is not null and
    away_deep_for_ema is not null and
    away_deep_against_ema is not null and
    home_ppda_for_ema is not null and
    home_ppda_against_ema is not null and
    away_ppda_for_ema is not null and
    away_ppda_against_ema is not null
    """
    
    return dwh_conn.export_to_pandas(v_sql)

def get_away_prediction_data(feature_type_filter, month_filter):
    
    v_sql = """
select
    football_match_his_lid,
    round(home_xg_against_ema,1)           home_xg_against_ema,
    round(away_xg_for_ema,1)               away_xg_for_ema
from
    betting_mart.ml_poisson_features
where
    --filter for iterator
    to_char(match_date,'yyyymm') = """ + str(month_filter) + """ and
    feature_type = '""" + str(feature_type_filter) + """' and
    home_goals_for_ema is not null and
    home_goals_against_ema is not null and
    away_goals_for_ema is not null and
    away_goals_against_ema is not null and
    home_xg_for_ema is not null and
    home_xg_against_ema is not null and
    away_xg_for_ema is not null and
    away_xg_against_ema is not null and
    home_shots_for_ema is not null and
    home_shots_against_ema is not null and
    away_shots_for_ema is not null and
    away_shots_against_ema is not null and
    home_sot_for_ema is not null and
    home_sot_against_ema is not null and
    away_sot_for_ema is not null and
    away_sot_against_ema is not null and
    home_corners_for_ema is not null and
    home_corners_against_ema is not null and
    away_corners_for_ema is not null and
    away_corners_against_ema is not null and
    home_deep_for_ema is not null and
    home_deep_against_ema is not null and
    away_deep_for_ema is not null and
    away_deep_against_ema is not null and
    home_ppda_for_ema is not null and
    home_ppda_against_ema is not null and
    away_ppda_for_ema is not null and
    away_ppda_against_ema is not null
    """
    
    return dwh_conn.export_to_pandas(v_sql)

## data function to get home and away data

In [4]:

import pandas
import numpy as np
from sklearn.datasets import make_blobs
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor


#truncate result table
#dwh_conn.execute('truncate table BETTING_DV.football_match_his_l_s_ml_poisson_expected_goals')


#get different feature types
df_feature_iterator = get_feature_types()

for index, row in df_feature_iterator.iterrows():
    print('Loop for feature type: ' + str(row['FEATURE_TYPE']))
    
    print('...getting data monthly loops for simulation')
    
    df_month_iterator = get_iterator_data(row['FEATURE_TYPE'])
    
    for index2, row2 in df_month_iterator.iterrows():
        
        print('...simulation for month ' + str(row2['ITERATOR']))
        
        print('.....getting training data')
        
        #get data to train home expGoals model
        df_home_train_data = get_home_training_data(row['FEATURE_TYPE'], row2['ITERATOR'])
        df_away_train_data = get_away_training_data(row['FEATURE_TYPE'], row2['ITERATOR'])
    
       #split features and pred classes
        df_home_train_x = df_home_train_data.iloc[:,:2]
        df_away_train_x = df_away_train_data.iloc[:,:2]
        df_train_home_y = df_home_train_data.iloc[:,2]
        df_train_away_y = df_away_train_data.iloc[:,2]
                
        print('......training models')
        
        print('......train home x: ' + str(df_home_train_x.shape))
        print('......train away x: ' + str(df_away_train_x.shape))
        print('......train home y: ' + str(df_train_home_y.shape))
        print('......train away y: ' + str(df_train_away_y.shape))
        

        
        model_home = RandomForestRegressor(
                                            max_depth = 5,
                                            #min_samples_leaf = 10,
                                            #min_samples_split = 150,
                                            #max_leaf_nodes = 20,
                                            n_estimators = 100,
                                            max_features = 2,
                                            random_state=0
                                            ).fit(df_home_train_x, df_train_home_y)
        
        
        model_away = RandomForestRegressor(
                                            max_depth = 4,
                                            #min_samples_leaf = 10,
                                            #min_samples_split = 150,
                                            #max_leaf_nodes = 20,
                                            n_estimators = 100,
                                            max_features = 2,
                                            random_state=0
                                            ).fit(df_away_train_x, df_train_away_y)
        


        print('......getting prediction data')
        
        #get data to train home expGoals model
        df_home_pred_data = get_home_prediction_data(row['FEATURE_TYPE'], row2['ITERATOR'])
        df_away_pred_data = get_away_prediction_data(row['FEATURE_TYPE'], row2['ITERATOR'])
        
        print('......df_home_pred_data: ' + str(df_home_pred_data.shape))
        print('......df_away_pred_data: ' + str(df_away_pred_data.shape))
        
        #do prediction
        df_home_pred_x = df_home_pred_data.iloc[:,1:3]
        df_away_pred_x = df_away_pred_data.iloc[:,1:3]
        
        print('......executing predition')  
    
        df_pred_home_expGoals = model_home.predict(df_home_pred_x)
        df_pred_away_expGoals = model_away.predict(df_away_pred_x)
    
        #create result data frame
        df_result_data = pd.DataFrame(df_home_pred_data.iloc[:,0:1])
        df_result_data['feature_type'] = 'random forest/xg/' + str(row['FEATURE_TYPE'])
        df_result_data['home_exp_goals'] = df_pred_home_expGoals
        df_result_data['away_exp_goals'] = df_pred_away_expGoals
        df_result_data['interator_param'] = str(row2['ITERATOR'])


        print(df_result_data.head())
        
        print('......data gets written to the db')    
        dwh_conn.import_from_pandas(df_result_data,('BETTING_DV','football_match_his_l_s_ml_poisson_expected_goals'))
    

Loop for feature type: EMA15 COMB
...getting data monthly loops for simulation
...simulation for month 201608
.....getting training data
......training models
......train home x: (3590, 2)
......train away x: (3590, 2)
......train home y: (3590,)
......train away y: (3590,)
......getting prediction data
......df_home_pred_data: (100, 3)
......df_away_pred_data: (100, 3)
......executing predition
             FOOTBALL_MATCH_HIS_LID                 feature_type  \
0  df74d6019d8ec8969da3b8a2d316d111  random forest/xg/EMA15 COMB   
1  733d356d163a75c04365593496d232e1  random forest/xg/EMA15 COMB   
2  acee56b53a1a434affc5cf8caedd2ef1  random forest/xg/EMA15 COMB   
3  6b2f710e0f9b6817cee287a160cfa86f  random forest/xg/EMA15 COMB   
4  fe382c3ac79fce6472a8d35927482d7f  random forest/xg/EMA15 COMB   

   home_exp_goals  away_exp_goals interator_param  
0        1.450289        0.977503          201608  
1        1.126793        1.211846          201608  
2        1.505419        1.266208   

...simulation for month 201704
.....getting training data
......training models
......train home x: (4960, 2)
......train away x: (4960, 2)
......train home y: (4960,)
......train away y: (4960,)
......getting prediction data
......df_home_pred_data: (280, 3)
......df_away_pred_data: (280, 3)
......executing predition
             FOOTBALL_MATCH_HIS_LID                 feature_type  \
0  45d9adc1ac85ebbc3ae282b4d0dea723  random forest/xg/EMA15 COMB   
1  8325fa391bcd8753c483e4b0e36c566f  random forest/xg/EMA15 COMB   
2  2b1217948b104826ac1ae44fc92f835c  random forest/xg/EMA15 COMB   
3  904cd484e4b3b00ec6b596b70c30a8ee  random forest/xg/EMA15 COMB   
4  295f4ef62dfa39d56fe2ed7db954fc51  random forest/xg/EMA15 COMB   

   home_exp_goals  away_exp_goals interator_param  
0        1.615379        0.821572          201704  
1        1.645914        1.311006          201704  
2        1.302391        0.965747          201704  
3        1.475783        1.326483          201704  
4        1.

...simulation for month 201802
.....getting training data
......training models
......train home x: (6489, 2)
......train away x: (6489, 2)
......train home y: (6489,)
......train away y: (6489,)
......getting prediction data
......df_home_pred_data: (192, 3)
......df_away_pred_data: (192, 3)
......executing predition
             FOOTBALL_MATCH_HIS_LID                 feature_type  \
0  5e9917fda3df324fedf6a04295e30c2d  random forest/xg/EMA15 COMB   
1  5eea0a2762973e4f5f685b5925f4ca98  random forest/xg/EMA15 COMB   
2  294ab534a68454aa256fbc49f7438136  random forest/xg/EMA15 COMB   
3  9e0520b3c9596568753131d5a1b7daf7  random forest/xg/EMA15 COMB   
4  e057d17ddda648f1a4ebab74653804c3  random forest/xg/EMA15 COMB   

   home_exp_goals  away_exp_goals interator_param  
0        1.366596        1.454709          201802  
1        1.602549        1.036107          201802  
2        1.546848        0.994993          201802  
3        1.853645        1.012572          201802  
4        1.

...simulation for month 201812
.....getting training data
......training models
......train home x: (7860, 2)
......train away x: (7860, 2)
......train home y: (7860,)
......train away y: (7860,)
......getting prediction data
......df_home_pred_data: (249, 3)
......df_away_pred_data: (249, 3)
......executing predition
             FOOTBALL_MATCH_HIS_LID                 feature_type  \
0  4552ce6db83c72975060f48d6893901c  random forest/xg/EMA15 COMB   
1  ad0d9c711b27657ab04b49d477d0793d  random forest/xg/EMA15 COMB   
2  462b7b50fa64886e8d93cf0b01b656c1  random forest/xg/EMA15 COMB   
3  cf6a150157e3e40b3fe5fcd9383143a9  random forest/xg/EMA15 COMB   
4  9892f50287f156de23f954402ca703e2  random forest/xg/EMA15 COMB   

   home_exp_goals  away_exp_goals interator_param  
0        1.531774        1.535483          201812  
1        1.656833        1.211674          201812  
2        1.394212        0.899280          201812  
3        1.258090        1.211674          201812  
4        1.

...simulation for month 201910
.....getting training data
......training models
......train home x: (9370, 2)
......train away x: (9370, 2)
......train home y: (9370,)
......train away y: (9370,)
......getting prediction data
......df_home_pred_data: (165, 3)
......df_away_pred_data: (165, 3)
......executing predition
             FOOTBALL_MATCH_HIS_LID                 feature_type  \
0  1fb31d814ca6f38ebf4b311d74d04ba8  random forest/xg/EMA15 COMB   
1  048fca762456c04edfeb4187c52b7919  random forest/xg/EMA15 COMB   
2  74171d939368be7ae22a703d33c55da0  random forest/xg/EMA15 COMB   
3  4dfe5158d80c5af72026430f938b0ad4  random forest/xg/EMA15 COMB   
4  e7733d20e21b86d90e04ebaf2b420be4  random forest/xg/EMA15 COMB   

   home_exp_goals  away_exp_goals interator_param  
0        2.011898        1.245361          201910  
1        2.200150        0.970334          201910  
2        1.879605        0.839733          201910  
3        1.153795        1.775028          201910  
4        1.

...

...simulation for month 201901
.....getting training data
......training models
......train home x: (8109, 2)
......train away x: (8109, 2)
......train home y: (8109,)
......train away y: (8109,)
......getting prediction data
......df_home_pred_data: (157, 3)
......df_away_pred_data: (157, 3)
......executing predition
             FOOTBALL_MATCH_HIS_LID                 feature_type  \
0  adb0e7e7eb2274454c45e24b49daeb2c  random forest/xg/EMA20 COMB   
1  eee6f64a88155aa963dea699fca8b6d3  random forest/xg/EMA20 COMB   
2  70c78eff6867997d49892c6f5614001d  random forest/xg/EMA20 COMB   
3  e5e8e44148fe947f74c131638678e92d  random forest/xg/EMA20 COMB   
4  5f0e84d53bd2fbe770e809401771517c  random forest/xg/EMA20 COMB   

   home_exp_goals  away_exp_goals interator_param  
0        2.822194        1.016289          201901  
1        1.269437        0.986769          201901  
2        1.239762        1.726438          201901  
3        2.372210        1.411797          201901  
4        1.

...simulation for month 201911
.....getting training data
......training models
......train home x: (9535, 2)
......train away x: (9535, 2)
......train home y: (9535,)
......train away y: (9535,)
......getting prediction data
......df_home_pred_data: (173, 3)
......df_away_pred_data: (173, 3)
......executing predition
             FOOTBALL_MATCH_HIS_LID                 feature_type  \
0  c3db7971b59642a4eb9a2c35ef81c239  random forest/xg/EMA20 COMB   
1  ecb70fb3cc801cf50cffa149f2638bb0  random forest/xg/EMA20 COMB   
2  d2fc32ba1bb37cb4138f80f59efece2d  random forest/xg/EMA20 COMB   
3  34ce9edafb82b7f892873d9ee013b179  random forest/xg/EMA20 COMB   
4  02684908673ab0466425dc6f5f9a4f15  random forest/xg/EMA20 COMB   

   home_exp_goals  away_exp_goals interator_param  
0        1.445409        1.132362          201911  
1        1.472266        1.362269          201911  
2        0.990972        0.993623          201911  
3        1.049259        2.448009          201911  
4        1.

...simulation for month 202008
.....getting training data
......training models
......train home x: (10751, 2)
......train away x: (10751, 2)
......train home y: (10751,)
......train away y: (10751,)
......getting prediction data
......df_home_pred_data: (26, 3)
......df_away_pred_data: (26, 3)
......executing predition
             FOOTBALL_MATCH_HIS_LID                 feature_type  \
0  704d49b64b74ec826e2f25da1f9681fa  random forest/xg/EMA20 COMB   
1  489b0191157951f163c0b014c83ec3f0  random forest/xg/EMA20 COMB   
2  41bbdd7cc4a483dd0027d1af0eb69b63  random forest/xg/EMA20 COMB   
3  cfade60557066d14c158ad25d6da1ec7  random forest/xg/EMA20 COMB   
4  fa703335fe1857101e81aa1d82d02c51  random forest/xg/EMA20 COMB   

   home_exp_goals  away_exp_goals interator_param  
0        2.239576        0.890867          202008  
1        0.971803        0.978022          202008  
2        1.284587        1.004377          202008  
3        1.653396        0.974919          202008  
4        

...simulation for month 202104
.....getting training data
......training models
......train home x: (12137, 2)
......train away x: (12137, 2)
......train home y: (12137,)
......train away y: (12137,)
......getting prediction data
......df_home_pred_data: (225, 3)
......df_away_pred_data: (225, 3)
......executing predition
             FOOTBALL_MATCH_HIS_LID                 feature_type  \
0  230f3c9370fb589ab89712b5a1adb012  random forest/xg/EMA20 COMB   
1  e9e9fdeafe74fa4e7bdf38c575384142  random forest/xg/EMA20 COMB   
2  b7ca1aab0ef300d17cc82aad6ab96ff6  random forest/xg/EMA20 COMB   
3  bc0af6f0f54c83cf36410f2c8c6754ee  random forest/xg/EMA20 COMB   
4  bf0e777c18079134bcde5cf6282894a0  random forest/xg/EMA20 COMB   

   home_exp_goals  away_exp_goals interator_param  
0        2.099202        0.818853          202104  
1        1.675581        1.695366          202104  
2        1.500810        0.984588          202104  
3        1.449821        1.164540          202104  
4      

...simulation for month 202202
.....getting training data
......training models
......train home x: (13631, 2)
......train away x: (13631, 2)
......train home y: (13631,)
......train away y: (13631,)
......getting prediction data
......df_home_pred_data: (200, 3)
......df_away_pred_data: (200, 3)
......executing predition
             FOOTBALL_MATCH_HIS_LID                 feature_type  \
0  08cbb222a63826c735a41c6586d7c1ea  random forest/xg/EMA20 COMB   
1  5e3aa55893484d632f0dd815ca3e2845  random forest/xg/EMA20 COMB   
2  9e78043eb2cba56efc3a2c1cb113203c  random forest/xg/EMA20 COMB   
3  fc38cbcdd3f730b50220d0d8061dc728  random forest/xg/EMA20 COMB   
4  192025f155f1998c2259240e1f2b1fca  random forest/xg/EMA20 COMB   

   home_exp_goals  away_exp_goals interator_param  
0        2.938032        1.150089          202202  
1        3.316251        0.972685          202202  
2        2.083303        1.461922          202202  
3        1.029531        1.824790          202202  
4      